# Float16 vs Float32 Speed Considerations
---

In [ ]:
import h5py
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime

The purpose of this notebook is to examine times for reading and writing hd5 files as full float32, truncated precision float 32, and short floats. 

# Generate Test Data

 Generate some nonsense data as combination of white noise and longer wavelength patterns. Generate truncated data with 4 signficant figures. 

In [31]:
nr, nc = 5000, 5000
size = (nr, nc)
x = np.arange(0, nc)
y = np.arange(0, nr)
X, Y = np.meshgrid(x, y)
image = 20*np.sin(X/60) + 20*np.sin(Y/200)
image[0:int(nc*.2),:] = 0  # zero out some to simulate padding in geocoded image.
myData = np.random.normal(loc=0, scale=20., size=size).astype(np.float32) + image.astype(np.float32)
start = datetime.now()
myDataShort = myData.astype(np.float16)
dt = datetime.now() - start
dtmsShort = float(dt.seconds) * 1000 + dt.microseconds / 1000
print(f'Time to short {dtmsShort} ms')
# Crude way to truncate, but serves its purpose
myDataTrunc = np.array([np.float32(np.format_float_positional(x, precision=4, fractional=False)) for x in myData.flatten()]).reshape(size)
dataSets = {'Float32': myData, 'Truncated': myDataTrunc, 'Float16': myDataShort}
#plt.imshow(myData)

Time to short 94.146 ms


Functions to read and write the data to an HDF5 file.

In [24]:
def writeHDF(data, filename, compression=None):
    if os.path.exists(filename):
        os.remove(filename)
    hf = h5py.File(filename, 'w')
    hf.create_dataset('test', data=data, compression=compression)
    hf.close()
    
def readHDF(filename):
    f = h5py.File(filename, 'r')
    a = np.array(f['test']).astype(np.float32)
    return a

## Write Test

Write the precomputed data sets to disk. This does not include the overhead of truncating precision or converting to float16 (~90ms in this case)

In [25]:
compressionLevel = 4
compressions = {'None': None, f'gzip{compressionLevel}': compressionLevel}
for dataSet in dataSets:
    for compression in compressions:
        start = datetime.now()
        filename =  f'{dataSet}.{compression}.h5'   
        writeHDF(dataSets[dataSet], filename, compression=compressions[compression])
        dt = datetime.now() - start
        dtms = float(dt.seconds) * 1000 + dt.microseconds / 1000.
        fileSize = os.path.getsize(filename)
        print(f'DataSet {dataSet:>10}, compression {compression:>10}, write time {dtms:5.1f} ms, fileSize {fileSize/1e6:.2f} MB')

DataSet    Float32, compression       None, write time 209.0 ms, fileSize 100.00 MB
DataSet    Float32, compression      gzip4, write time 3515.8 ms, fileSize 92.17 MB
DataSet  Truncated, compression       None, write time 191.1 ms, fileSize 100.00 MB
DataSet  Truncated, compression      gzip4, write time 3193.0 ms, fileSize 74.40 MB
DataSet    Float16, compression       None, write time 102.8 ms, fileSize 50.00 MB
DataSet    Float16, compression      gzip4, write time 1536.3 ms, fileSize 45.49 MB


Now read back. Note the data are converted back to float 32, so times are directly comparable.

In [26]:
for dataSet in dataSets:
    for compression in compressions:
        filename =  f'{dataSet}.{compression}.h5'
        start = datetime.now()
        data = readHDF(filename)
        dt = datetime.now() - start
        dtms = float(dt.seconds) * 1000 + dt.microseconds / 1000.
        fileSize = os.path.getsize(filename)
        print(f'DataSet {dataSet:>10}, compression {compression:>10}, read time {dtms:5.1f} ms, fileSize {fileSize/1e6:.2f} MB, error {np.std(dataSets["Float32"]-data):.5f}')
        if os.path.exists(filename):
            os.remove(filename)

DataSet    Float32, compression       None, read time 142.8 ms, fileSize 100.00 MB, error 0.00000
DataSet    Float32, compression      gzip4, read time 579.2 ms, fileSize 92.17 MB, error 0.00000
DataSet  Truncated, compression       None, read time 141.6 ms, fileSize 100.00 MB, error 0.00245
DataSet  Truncated, compression      gzip4, read time 821.1 ms, fileSize 74.40 MB, error 0.00245
DataSet    Float16, compression       None, read time 117.4 ms, fileSize 50.00 MB, error 0.00555
DataSet    Float16, compression      gzip4, read time 396.5 ms, fileSize 45.49 MB, error 0.00555


# Summary

The following conclusions can be drawn:
- The conversion to float16 is about ~100 ms.
- For a not fast disk, conversion time is mostly made up on faster write.
- Compression slows read/write considerably in all cases.
- Truncation to 4 digits buys some space savings, but only about half has much as float16. Compression doesn't buy much otherwise (could be more with more background).

The bottom line is short floats seem to work well and add no substantial overhead due to conversion and in some cases save time on i/o.

Intel processors have support for the conversion since 2013. If performance is slow on other systems it could be because the compiler is not using the special instructions. 